In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from surface_seg.envs.multicomponent_surface_env import MultiComponentSurface

MCS_gym = MultiComponentSurface()

In [ ]:
import tensorforce 
from tensorforce.agents import Agent


environment = tensorforce.environments.OpenAIGym(MCS_gym,
                                                 max_episode_timesteps=200)

agent = Agent.create(
    agent='dqn', environment=environment, 
    batch_size=10, 
    learning_rate=1e-3,
    memory = 1000,
    max_episode_timesteps = 200
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent,
    environment=environment,
    max_episode_timesteps=200
)

runner.run(num_episodes=200)

runner.run(num_episodes=100, evaluation=True)

runner.close()